# data to blob


In [1]:
#read in varaibles from other notebook
%store -r hashmap
print(hashmap)

{'18699': '..\\data\\flow\\task_177_1627319652\\sensor_18699\\sensor_measures_20210717_20210721_1.csv', '18720': '..\\data\\flow\\task_177_1627319652\\sensor_18720\\sensor_measures_20210711_20210721_1.csv', '18749': '..\\data\\flow\\task_177_1627319652\\sensor_18749\\sensor_measures_20210716_20210721_1.csv'}


In [2]:
%store -r sensorIds
print(sensorIds)

['18699', '18720', '18749']


In [3]:
import pandas as pd
import numpy as np
import psycopg2
import json

# Intialising the measurement dataframe

In [4]:
#bringing the csv to pandas dataframes
dfList = []
for snum in sensorIds:
    dfList.append(pd.read_csv(hashmap[snum],parse_dates=True, index_col="timestamp"))

In [5]:
#put the dataframes into a dictionary with the sensor id as the key
dataframes = {k:v for k,v in zip(sensorIds,dfList)}
dataframes[sensorIds[0]].head()

,date,NO2 (ppb),NO2 (Plume AQI),VOC (ppb),VOC (Plume AQI),pm 1 (ug/m3),pm 1 (Plume AQI),pm 10 (ug/m3),pm 10 (Plume AQI),pm25 (ug/m3),pm 25 (Plume AQI)
timestamp,,,,,,,,,,,
1626501264,2021-07-17 05:54:24,0,0,63,5,6,15,82,102,12,23
1626501324,2021-07-17 05:55:24,2,2,56,4,3,8,5,5,4,9
1626501384,2021-07-17 05:56:24,5,4,51,4,3,8,5,5,4,8
1626501444,2021-07-17 05:57:24,0,0,47,4,3,8,5,5,4,8
1626501504,2021-07-17 05:58:24,2,2,45,4,4,11,43,43,6,12


In [6]:
# drop date column and add sensor id column (foreign key)
for key in dataframes:
    df = dataframes[key]
    df.drop("date", axis=1, inplace=True)

    #inserting row into temp array
    #tempyArray = [timestamp_start,timestamp_end,sensor_id,jsonString]
    tempArray = [df.index[0],df.index[-1],int(key),df.to_json(orient="index")] 

    dataframes[key] = tempArray    #assign new dataframe to coressponding key

In [7]:
# reading the JSON data using json.loads(json string)
# converting json dataset from dictionary to dataframe
dict_data = json.loads(dataframes[sensorIds[0]][3])
data = pd.DataFrame.from_dict(dict_data, orient='index')
data.index.rename('timestamp', inplace=True)
data.head()

,NO2 (ppb),NO2 (Plume AQI),VOC (ppb),VOC (Plume AQI),pm 1 (ug/m3),pm 1 (Plume AQI),pm 10 (ug/m3),pm 10 (Plume AQI),pm25 (ug/m3),pm 25 (Plume AQI)
timestamp,,,,,,,,,,
1626501264,0,0,63,5,6,15,82,102,12,23
1626501324,2,2,56,4,3,8,5,5,4,9
1626501384,5,4,51,4,3,8,5,5,4,8
1626501444,0,0,47,4,3,8,5,5,4,8
1626501504,2,2,45,4,4,11,43,43,6,12


# Exporting to PostgresSQL 


# Writing records from a Dictionary of arrays to a SQL database
loop over all the keys and execute insert query

In [8]:
#Connecting to an existing database
con = psycopg2.connect(
    host="localhost",
    database="airQuality",
    user="Riyad", 
    password="123",
    # attempt to connect for 3 seconds then raise exception
    connect_timeout = 3)

In [9]:
#Opening a cursor to execute database operations
cursor = con.cursor()
query = "SELECT * FROM sensor_network.sensors"
#change dataframe to csv and save file
sensorsdf = pd.read_sql_query(query, con, index_col='plume_id')

cursor.close()

In [10]:
#sensorsdf.head()

,sensor_serial_number,id,type_id
plume_id,,,
17539,02:00:00:00:43:8b,1,1
19115,02:00:00:00:49:b3,2,1
17521,02:00:00:00:43:79,3,1
16397,02:00:00:00:3f:15,4,1
16701,02:00:00:00:40:45,5,1


In [11]:
for key in dataframes:
    #get the new key from sensors table
    sensor_id = sensorsdf.loc[int(key)][1]
    try:
        #Opening a cursor to execute database operations
        cursor = con.cursor()
        cursor.execute("INSERT INTO sensor_data.sensor_measurements (timestamp_start,timestamp_end, sensor_id, measurements) VALUES(%s, %s, %s, %s)", (int(dataframes[key][0]), int(dataframes[key][1]), int(sensor_id),str(dataframes[key][3])))
        con.commit() 
        cursor.close()
    #if table name does not exist exit loop 
    except(psycopg2.errors.UndefinedTable) as error:
        print('ERROR: ' + error)
        break

In [12]:
#closing the connection
con.close()